In [1]:
import torch, torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

from bgan.cnnTrainer import CnnTrainer
from bgan.wcnnTrainer import WcnnTrainer
from bgan.vatCnnTrainer import VatCnnTrainer
from bgan.networkparts import *
import bgan.augLayers as augLayers
%matplotlib inline
%load_ext autoreload
%autoreload 2

## CNN Configuration

In [5]:
savedir = '/home/maf388/tb-experiments/layer13wAugTransf2/'
config = {'base_lr':1e-4, 'amntLab':4000, 
          'lab_BS':32}

baseCNN = layer13(numClasses=10)

fullCNN = nn.Sequential(
#     augLayers.RandomTranslate(4),
#     augLayers.RandomHorizontalFlip(),
    augLayers.GaussianNoise(0.15),
    baseCNN,
)

## Dataset

In [6]:
img_size = 32
transform_dev = transforms.Compose(
    [transforms.Resize(img_size),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_train = transform_dev
transform_train = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.RandomHorizontalFlip(),
     transform_dev])

pathToDataset = '/scratch/datasets/cifar10/'
trainset = CIFAR10(pathToDataset, download=True, transform=transform_train)
devset = CIFAR10(pathToDataset, train=False, download=True, transform=transform_dev)
testset = None

datasets = (trainset, devset, testset)

Files already downloaded and verified
Files already downloaded and verified


In [21]:
trainer = CnnTrainer(fullCNN,datasets,savedir,**config)

9.64 s ± 244 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
trainer.train(640)

[  0/640][     0/80000] Val_Acc: 0.100 Train_Acc(Batch): 0.031 
Traceback (most recent call last):
  File "/home/maf388/.conda/envs/env0/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-0efafca1f2eb>", line 1, in <module>
    trainer.train(640)
  File "/home/maf388/bayesganfuture/bgan/cnnTrainer.py", line 68, in train
    trainData = to_var_gpu(next(self.train_iter))
  File "/home/maf388/bayesganfuture/bgan/utils.py", line 12, in to_var_gpu
    return type(x)(map(to_var_gpu, x))
  File "/home/maf388/bayesganfuture/bgan/utils.py", line 14, in to_var_gpu
    return Variable(x).cuda()
  File "/home/maf388/.conda/envs/env0/lib/python3.5/site-packages/torch/autograd/variable.py", line 298, in cuda
    return CudaTransfer.apply(self, device, async)
  File "/home/maf388/.conda/envs/env0/lib/python3.5/site-packages/torch/autograd/_functions/tensor.py", line 201, in forward
    r

KeyboardInterrupt: 

In [ ]:
trainer.CNN.training